# All of the code and documentation below was generated using a Large Language Model - Claude Opus. See github for the full chat transcript.
Aidan Riordan
# Oil Well Maintenance Optimization using Basic Variable Neighborhood Search (BVNS)


This Jupyter Notebook demonstrates the implementation and application of the Basic Variable Neighborhood Search (BVNS) algorithm for optimizing the scheduling of maintenance services for oil wells.  The objective is to minimize the total loss, which is calculated as the sum of the product of daily oil production and the completion time of maintenance service for each well.

## Problem Description

The problem involves a set of oil wells and a fleet of rigs that perform maintenance services on the wells. Each well has a specific daily oil production and requires a certain duration for maintenance service. The travel times between wells and from the initial rig positions to the wells are known.

The goal is to assign the wells to the rigs and determine the optimal sequence of maintenance services for each rig to minimize the total loss.

## BVNS Algorithm

The BVNS algorithm is a metaheuristic optimization technique that explores different neighborhood structures to find high-quality solutions. The algorithm consists of the following main steps:

1. Initialization: Generate an initial random solution.
2. Shaking: Apply a random neighborhood structure to the current solution to escape from local optima.
3. Local Search: Perform a local search to improve the solution obtained from the shaking step.
4. Move or Not: If the local search improves the solution, update the best solution and reset the neighborhood structure. Otherwise, move to the next neighborhood structure.
5. Termination: Repeat steps 2-4 until a specified time limit is reached.

## Neighborhood Structures

The BVNS algorithm utilizes the following neighborhood structures:

1. Swap routes (SR): The wells and the associated routes assigned to two workover rigs are swapped. Each solution has m(m − 1)/2 neighbors within this neighborhood.

2. Swap wells from the same workover rig (SWSW): The order in which two wells are serviced by the same workover rig is swapped. Assuming that the n wells are evenly assigned to the m workover rigs, each solution has n(n−m)/(2m) neighbors within this neighborhood.

3. Swap wells from different workover rigs (SWDW): Two wells assigned to two different workover rigs are swapped. Once again assuming that the n wells are evenly assigned to the m workover rigs, each solution has n^2(m − 1)/(2m) neighbors within this neighborhood.

4. Add-Drop (AD): A well assigned to a workover rig is reassigned to any position of the schedule of another workover rig. Once again assuming that the n wells are evenly assigned to the m rigs, each solution has also n^2(m − 1)/(2m) neighbors within this neighborhood.

Five other neighborhoods are defined by successive applications of moves within neighborhoods SWSW, SWDW, and AD:

5. SWSW2: Successively apply two moves within neighborhood SWSW.
6. SWDW2: Successively apply two moves within neighborhood SWDW.
7. SWDW3: Successively apply three moves within neighborhood SWDW.
8. AD2: Successively apply two moves within neighborhood AD.
9. AD3: Successively apply three moves within neighborhood AD.

## Variables

The following variables are used in the code:

- `n`: The number of oil wells.
- `m`: The number of rigs.
- `p`: A list representing the daily oil production of each well.
- `d`: A list representing the maintenance service duration for each well.
- `t`: A matrix representing the travel times between wells.
- `e`: A matrix representing the travel times from the initial rig positions to each well.
- `x`: A list representing the starting times of maintenance services for each well.
- `y`: A list of lists representing the assignment of wells to rigs.
- `time_limit`: The time limit for the BVNS algorithm.
- `kmax`: The maximum number of neighborhood structures to explore.

## Usage

To use this notebook, simply run the cells in sequential order. The notebook will generate a random problem instance based on the specified parameters and apply the BVNS algorithm to find the optimal solution.

You can modify the problem parameters, such as the number of wells (`n`), number of rigs (`m`), and the time limit for the algorithm (`time_limit`), to experiment with different scenarios.

The notebook provides detailed comments throughout the code to explain the functionality of each function and section.

## Conclusion

This notebook demonstrates the application of the Basic Variable Neighborhood Search algorithm for optimizing the scheduling of maintenance services for oil wells. By exploring different neighborhood structures and performing local search, the algorithm aims to find high-quality solutions that minimize the total loss.

The notebook serves as a starting point for understanding and implementing the BVNS algorithm for similar optimization problems. Feel free to modify and extend the code to suit your specific requirements and explore further enhancements to the algorithm.

In [1]:
import random
import time

random.seed(5)

def bvns(n, m, p, d, t, e, time_limit, kmax):
    start_time = time.time()
    
    # Initialize the solution
    x, y = initialize_solution(n, m, d, t, e)
    best_solution = (x, y)
    best_loss = objective_function(x, y, p, d)

    print(f"Initial solution: Objective function value = {best_loss}")

    iteration = 1
    while time.time() - start_time < time_limit:
        print(f"\nIteration {iteration}:")
        k = 1
        while k <= kmax:
            # Shake
            x_prime, y_prime = shake(x, y, k, m)
            loss_shake = objective_function(x_prime, y_prime, p, d)
            print(f"  Shake (k = {k}): Objective function value = {loss_shake}")

            # Local search
            x_local, y_local = local_search(x_prime, y_prime, p, d, t, e)
            loss_local = objective_function(x_local, y_local, p, d)
            print(f"  Local search: Objective function value = {loss_local}")

            # Move or not
            if loss_local < best_loss:
                best_solution = (x_local, y_local)
                best_loss = loss_local
                print(f"  Improved solution found: Objective function value = {best_loss}")
                k = 1
            else:
                k += 1

        iteration += 1

    return best_solution, best_loss

def initialize_solution(n, m, d, t, e):
    x = [0] * n
    y = [[] for _ in range(m)]

    wells = list(range(n))
    random.shuffle(wells)

    for well in wells:
        rig = random.randint(0, m - 1)
        y[rig].append(well)
    
    # Update the starting times based on the initial well assignments
    x = update_starting_times(y, d, t, e)

    return x, y

def shake(x, y, k, m):
    # Apply different neighborhood structures based on the value of k
    if k == 1:
        return swap_routes(x, y, m)
    elif k == 2:
        return swap_wells_same_rig(x, y, m)
    elif k == 3:
        return swap_wells_different_rigs(x, y, m)
    elif k == 4:
        return add_drop(x, y, m)
    elif k == 5:
        # SWSW2: successively apply two moves within neighborhood SWSW
        x_prime, y_prime = swap_wells_same_rig(x, y, m)
        x_prime, y_prime = swap_wells_same_rig(x_prime, y_prime, m)
        return x_prime, y_prime
    elif k == 6:
        # SWDW2: successively apply two moves within neighborhood SWDW
        x_prime, y_prime = swap_wells_different_rigs(x, y, m)
        x_prime, y_prime = swap_wells_different_rigs(x_prime, y_prime, m)
        return x_prime, y_prime
    elif k == 7:
        # SWDW3: successively apply three moves within neighborhood SWDW
        x_prime, y_prime = swap_wells_different_rigs(x, y, m)
        x_prime, y_prime = swap_wells_different_rigs(x_prime, y_prime, m)
        x_prime, y_prime = swap_wells_different_rigs(x_prime, y_prime, m)
        return x_prime, y_prime
    elif k == 8:
        # AD2: successively apply two moves within neighborhood AD
        x_prime, y_prime = add_drop(x, y, m)
        x_prime, y_prime = add_drop(x_prime, y_prime, m)
        return x_prime, y_prime
    elif k == 9:
        # AD3: successively apply three moves within neighborhood AD
        x_prime, y_prime = add_drop(x, y, m)
        x_prime, y_prime = add_drop(x_prime, y_prime, m)
        x_prime, y_prime = add_drop(x_prime, y_prime, m)
        return x_prime, y_prime
    else:
        raise ValueError(f"Invalid neighborhood structure: {k}")

def swap_routes(x, y, m):
    x_prime = x.copy()
    y_prime = [route.copy() for route in y]

    # Randomly select two different rigs
    rig1, rig2 = random.sample(range(m), 2)

    # Swap the routes of the selected rigs
    y_prime[rig1], y_prime[rig2] = y_prime[rig2], y_prime[rig1]
    
    #Ensure soln is feasable
    x_prime = update_starting_times(y_prime, d, t, e)

    return x_prime, y_prime

def swap_wells_same_rig(x, y, m):
    x_prime = x.copy()
    y_prime = [route.copy() for route in y]

    # Randomly select a rig
    rig = random.randint(0, m - 1)

    # Randomly select two different wells from the same rig
    if len(y_prime[rig]) < 2:
        return x_prime, y_prime

    well1, well2 = random.sample(range(len(y_prime[rig])), 2)

    # Swap the positions of the selected wells within the rig's route
    y_prime[rig][well1], y_prime[rig][well2] = y_prime[rig][well2], y_prime[rig][well1]
    
    # Ensure soln is feasable
    x_prime = update_starting_times(y_prime, d, t, e)
    
    return x_prime, y_prime

def swap_wells_different_rigs(x, y, m):
    x_prime = x.copy()
    y_prime = [route.copy() for route in y]

    # Randomly select two different rigs
    rig1, rig2 = random.sample(range(m), 2)

    # Randomly select a well from each rig
    if len(y_prime[rig1]) == 0 or len(y_prime[rig2]) == 0:
        return x_prime, y_prime

    well1 = random.randint(0, len(y_prime[rig1]) - 1)
    well2 = random.randint(0, len(y_prime[rig2]) - 1)

    # Swap the selected wells between the rigs
    y_prime[rig1][well1], y_prime[rig2][well2] = y_prime[rig2][well2], y_prime[rig1][well1]
    x_prime = update_starting_times(y_prime, d, t, e)

    return x_prime, y_prime

def add_drop(x, y, m):
    x_prime = x.copy()
    y_prime = [route.copy() for route in y]

    # Randomly select a well and a rig
    well = random.randint(0, len(x_prime) - 1)
    rig_from = None
    for i in range(m):
        if well in y_prime[i]:
            rig_from = i
            break

    rig_to = random.randint(0, m - 1)
    while rig_to == rig_from:
        rig_to = random.randint(0, m - 1)

    # Remove the well from the source rig
    y_prime[rig_from].remove(well)

    # Insert the well at a random position in the destination rig
    insert_pos = random.randint(0, len(y_prime[rig_to]))
    y_prime[rig_to].insert(insert_pos, well)

    # Update the starting times based on the new well assignments
    x_prime = update_starting_times(y_prime, d, t, e)

    return x_prime, y_prime

def local_search(x, y, p, d, t, e):
    x_local = x.copy()
    y_local = [route.copy() for route in y]

    improved = True
    while improved:
        improved = False
        for rig1 in range(len(y_local)):
            for well1 in range(len(y_local[rig1])):
                for rig2 in range(len(y_local)):
                    for well2 in range(len(y_local[rig2])):
                        if rig1 != rig2 or well1 != well2:
                            # Swap the wells
                            y_local[rig1][well1], y_local[rig2][well2] = y_local[rig2][well2], y_local[rig1][well1]

                            # Update the starting times based on the new well assignments
                            x_local = update_starting_times(y_local, d, t, e)

                            # Evaluate the objective function after the swap
                            loss_local = objective_function(x_local, y_local, p, d)
                            loss_current = objective_function(x, y, p, d)

                            if loss_local < loss_current:
                                x = x_local.copy()
                                y = [route.copy() for route in y_local]
                                improved = True
                            else:
                                # Revert the swap if the solution did not improve
                                y_local[rig1][well1], y_local[rig2][well2] = y_local[rig2][well2], y_local[rig1][well1]
                                x_local = update_starting_times(y_local, d, t, e)

    return x, y

def update_starting_times(y, d, t, e):
    x = [0] * len(d)
    for rig in range(len(y)):
        for i in range(len(y[rig])):
            well = y[rig][i]
            if i == 0:
                x[well] = e[rig][well]
            else:
                prev_well = y[rig][i - 1]
                x[well] = x[prev_well] + d[prev_well] + t[prev_well][well]
    return x

def objective_function(x, y, p, d):
    n = len(p)  # number of wells
    m = len(y)  # number of rigs
    total_loss = 0
    for j in range(n):
        total_loss += p[j] * (x[j] + d[j])
    return total_loss

    #One line improvement from new chat 
    #return sum(p[j] * (x[j] + d[j]) for j in range(len(p)))  



In [ ]:
#Generate rand problem instance
def generate_problem_instance(n, m):
    # Generate daily oil production for each well
    p = [random.randint(1, 100) for _ in range(n)]

    # Generate maintenance service duration for each well
    d = [random.randint(1, 5) for _ in range(n)]

    # Generate travel time between wells
    t = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            time = random.randint(1, 100)
            t[i][j] = time
            t[j][i] = time

    # Generate travel time from initial rig position to each well
    e = [[random.randint(1, 100) for _ in range(n)] for _ in range(m)]

    return p, d, t, e

In [3]:
#Generate rand problem instance
n = 130
m = 9
p, d, t, e = generate_problem_instance(n, m)


In [4]:
# Run BVNS on generated instance
time_limit = 600 #~1 min note may take longer
kmax = 9

best_solution, best_loss = bvns(n, m, p, d, t, e, time_limit, kmax)


Initial solution: Objective function value = 2740438

Iteration 1:
  Shake (k = 1): Objective function value = 2788078
  Local search: Objective function value = 361022
  Improved solution found: Objective function value = 361022
  Shake (k = 1): Objective function value = 2676503
  Local search: Objective function value = 416076
  Shake (k = 2): Objective function value = 2749727
  Local search: Objective function value = 382665
  Shake (k = 3): Objective function value = 2723608
  Local search: Objective function value = 402442
  Shake (k = 4): Objective function value = 2711136
  Local search: Objective function value = 419415
  Shake (k = 5): Objective function value = 2711043
  Local search: Objective function value = 370485
  Shake (k = 6): Objective function value = 2791769
  Local search: Objective function value = 449348
  Shake (k = 7): Objective function value = 2697020
  Local search: Objective function value = 404282
  Shake (k = 8): Objective function value = 2744154
  Lo

In [5]:
print("Best solution:")
print("x =", best_solution[0])
print("y =", best_solution[1], "\n")
print("Best loss BVNS:", best_loss)

Best solution:
x = [21, 41, 51, 87, 52, 10, 99, 38, 130, 32, 56, 66, 4, 22, 19, 8, 41, 65, 110, 33, 17, 117, 18, 103, 161, 34, 144, 92, 58, 88, 84, 24, 141, 66, 127, 84, 45, 3, 19, 27, 93, 180, 153, 62, 58, 31, 4, 138, 72, 88, 67, 46, 78, 10, 139, 106, 116, 64, 76, 91, 66, 21, 9, 6, 134, 61, 90, 107, 76, 31, 24, 170, 26, 89, 83, 90, 146, 58, 73, 113, 13, 25, 73, 46, 42, 53, 58, 73, 42, 47, 202, 37, 18, 33, 147, 72, 91, 95, 76, 63, 52, 68, 184, 1, 40, 53, 83, 120, 67, 97, 36, 97, 73, 99, 14, 3, 51, 58, 93, 35, 30, 1, 8, 131, 125, 101, 15, 31, 25, 46]
y = [[15, 126, 81, 127, 25, 89, 4, 28, 57, 52, 106, 59, 23, 18, 21, 34, 94, 24, 41], [46, 72, 93, 10, 43, 101, 112, 29, 97], [121, 62, 61, 120, 104, 36, 99, 108, 95, 98, 3, 96, 125, 55, 107, 8, 26, 71], [103, 12, 122, 92, 31, 7, 84, 100, 44, 33, 82, 30, 66, 118, 79, 64, 76], [53, 38, 69, 88, 51, 65, 17, 50, 58, 40, 113], [37, 80, 14, 70, 91, 2, 117, 11, 68, 74, 49, 27, 6, 54], [115, 22, 128, 19, 1, 105, 48, 73, 109], [63, 114, 0, 45, 119, 1

In [6]:
#Monte carlo new chat
import random
import time

def monte_carlo(n, m, p, d, t, e, time_limit):
    start_time = time.time()
    
    # Initialize the best solution
    best_solution = initialize_solution(n, m, d, t, e)
    best_loss = objective_function(best_solution[0], best_solution[1], p, d)

    while time.time() - start_time < time_limit:
        # Randomly generate a solution
        solution = initialize_solution(n, m, d, t, e)
        loss = objective_function(solution[0], solution[1], p, d)

        # Update the best solution if the current solution is better
        if loss < best_loss:
            best_solution = solution
            best_loss = loss

    return best_solution, best_loss

In [7]:
best_solution_mont, best_loss_mont = monte_carlo(n, m, p, d, t, e, time_limit)

In [8]:
print("Best loss monte carlo:", best_loss_mont)

Best loss monte carlo: 1930719


# # Problem instance example hardcoded


In [9]:
# Problem instance example
n = 10  # number of wells
m = 3  # number of rigs
p = [10, 2, 12, 6, 90, 8, 15, 4, 7, 11]  # daily oil production of each well

d = [3, 2, 4, 2, 3, 2, 4, 3, 2, 3]  # maintenance service duration for each well
t = [[0, 20, 3, 1, 200, 30, 10, 5, 15, 25],
     [20, 0, 2, 3, 1, 25, 15, 10, 8, 12],
     [3, 2, 0, 2, 3, 20, 12, 8, 6, 10],
     [1, 3, 2, 0, 2, 18, 10, 6, 4, 8],
     [200, 1, 3, 2, 0, 220, 210, 205, 202, 198],
     [30, 25, 20, 18, 220, 0, 10, 15, 20, 25],
     [10, 15, 12, 10, 210, 10, 0, 5, 8, 12],
     [5, 10, 8, 6, 205, 15, 5, 0, 3, 7],
     [15, 8, 6, 4, 202, 20, 8, 3, 0, 5],
     [25, 12, 10, 8, 198, 25, 12, 7, 5, 0]]  # travel time between wells
e = [[100, 2, 3, 2, 1, 120, 90, 80, 70, 60],
     [2, 1, 2, 3, 2, 30, 25, 20, 15, 10],
     [3, 2, 1, 2, 3, 35, 30, 25, 20, 15]]  # travel time from initial rig position to each well

# BVNS Function

"The function takes the necessary input parameters and initializes the start time.
It initializes the solution using the initialize_solution function and evaluates the initial solution using the objective_function.
The main loop runs until the time limit is reached. In each iteration:

- It initializes the neighborhood structure counter k to 1.
- It enters an inner loop that runs until k exceeds kmax.
- Inside the inner loop:
  - It performs a shake using the shake function based on the current value of k.
  - It performs a local search using the local_search function to improve the solution.
  - It compares the loss of the local search solution with the best loss found so far.
  - If the local search solution is better, it updates the best solution and best loss, and resets k to 1.
  - If the local search solution is not better, it increments k to move to the next neighborhood structure.

- It increments the iteration counter.

Finally, it returns the best solution found and the corresponding best loss value."

# Swap Routes Function

"Here's how the function works:

- It creates copies of the current starting times (x_prime) and well assignments (y_prime) to avoid modifying the original solution.
- It randomly selects two different rigs (rig1 and rig2) using random.sample() from the range of available rigs (range(m)).
- It swaps the routes of the selected rigs by exchanging the well assignments between y_prime[rig1] and y_prime[rig2].
- It ensures the feasibility of the new solution by updating the starting times of the wells based on the new well assignments using the update_starting_times function.
- Finally, it returns the updated starting times (x_prime) and well assignments (y_prime)."

# Swap Wells Between Different Rigs

"Here's how the function works:

- It creates copies of the current starting times (x_prime) and well assignments (y_prime) to avoid modifying the original solution.
- It randomly selects two different rigs (rig1 and rig2) using random.sample() from the range of available rigs (range(m)).
- It checks if either of the selected rigs has no wells assigned to it. If any of the rigs is empty, it immediately returns the current solution without making any changes.
- If both rigs have wells assigned, it randomly selects a well from each rig (well1 from rig1 and well2 from rig2) using random.randint().
- It swaps the selected wells between the rigs by exchanging y_prime[rig1][well1] and y_prime[rig2][well2].
- It updates the starting times of the wells based on the new well assignments using the update_starting_times function.
- Finally, it returns the updated starting times (x_prime) and well assignments (y_prime).

The function appears to be working correctly. It successfully swaps wells between two randomly selected rigs and updates the starting times accordingly to maintain the feasibility of the solution."

# Add-Drop Function

"The function takes the following parameters:

- x: The current starting times of the wells.
- y: The current well assignments to rigs.
- m: The number of rigs.

Here's how the function works:

- It creates copies of the current starting times (x_prime) and well assignments (y_prime) to avoid modifying the original solution.
- It randomly selects a well (well) from the range of available wells (0 to len(x_prime) - 1).
- It finds the rig (rig_from) that the selected well belongs to by iterating through the rigs and checking if the well is present in any of them. It breaks the loop once the rig is found.
- It randomly selects a destination rig (rig_to) from the range of available rigs (0 to m - 1). If the destination rig is the same as the source rig, it keeps selecting a new destination rig until they are different.
- It removes the selected well from the source rig using y_prime[rig_from].remove(well).
- It randomly selects an insertion position (insert_pos) in the destination rig using random.randint(0, len(y_prime[rig_to])).
- It inserts the well at the selected position in the destination rig using y_prime[rig_to].insert(insert_pos, well).
- It updates the starting times of the wells based on the new well assignments using the update_starting_times function.
- Finally, it returns the updated starting times (x_prime) and well assignments (y_prime)."

# Update Starting Times Function

"The function takes the following parameters:

- y: The current well assignments to rigs.
- d: The maintenance service duration for each well.
- t: The travel time between wells.
- e: The travel time from the initial rig position to each well.

Here's how the function works:

- It initializes a list x with zeros, where the length of x is equal to the number of wells (determined by len(d)). This list will store the updated starting times for each well.
- It iterates over each rig in y using the rig variable.
- For each rig, it iterates over the wells assigned to that rig using the i variable.
- For each well, it retrieves the well index using well = y[rig][i].
- If the well is the first well in the rig's route (i.e., i == 0), it sets the starting time of the well to the travel time from the initial rig position to that well, which is stored in e[rig][well].
- If the well is not the first well in the rig's route, it retrieves the previous well index using prev_well = y[rig][i - 1].
- It calculates the starting time of the current well by adding the starting time of the previous well (x[prev_well]), the maintenance service duration of the previous well (d[prev_well]), and the travel time from the previous well to the current well (t[prev_well][well]).
- After iterating over all the wells and rigs, it returns the updated starting times stored in x.